In [2]:
!pip install gradio pillow opencv-python -U deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import requests
import json
import base64
import io
from PIL import Image
import cv2
import numpy as np
from deep_translator import GoogleTranslator


url_generate = "URL_OF_FLUX_SCHNELL_API"
url_inpaint_and_outpaint = "URL_OF_FLUX_FILL_DEV_API"

neg_prompt = "blurry, low resolution, distorted, deformed, ugly, bad anatomy, bad proportions, extra limbs, fused fingers, missing fingers, mutated hands, long neck, duplicate face, extra face, poorly drawn face, out of frame, cropped, watermark, signature, text, error, jpeg artifacts, unnatural skin, unrealistic colors, grainy, overexposed, underexposed, bad lighting, wrong perspective"

def translate_text(text):
     return str(GoogleTranslator(source='auto', target='en').translate(text))

def encode_image(input_image: Image.Image) -> str:
    # Save to a temporary in-memory file instead of disk
    buffered = io.BytesIO()
    input_image.save(buffered, format="PNG")  # or "JPEG", depending on the image
    encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return encoded_string

def decode_result(result):
    encoded_image_in_base64 = json.loads(result.text)['image']
    decoded_bytes = base64.b64decode(encoded_image_in_base64)
    image = Image.open(io.BytesIO(decoded_bytes))
    return image

def send_api_call(payload, url):
    result = requests.post(
        url = url,
        data = json.dumps(payload)
    )
    return result

def get_inpaint_image_and_mask_and_size(image_and_mask):
    image = np.asarray(image_and_mask['background'])
    alpha_mask = np.asarray(image_and_mask['layers'][0])
    _, mask = cv2.threshold(alpha_mask[:, :, 3], 0, 255, cv2.THRESH_BINARY)

    image = Image.fromarray(image)
    mask = Image.fromarray(mask)
    return image,mask,image.size


def get_outpaint_image_and_mask_and_size(input_image, ratio):
    old_im = Image.fromarray(input_image)
    old_size = old_im.size

    new_width = old_size[0] + round((ratio * old_size[0]) / 100)
    new_height = old_size[1] + round((ratio * old_size[1]) / 100)
    new_size = (new_width, new_height)

    new_im = Image.new("RGB", new_size, "White")
    box = tuple((n - o) // 2 for n, o in zip(new_size, old_size))
    new_im.paste(old_im, box)

    mask = Image.new("L", old_size, "Black")
    mask_enlarged = Image.new("L", new_size, "White")
    mask_enlarged.paste(mask, box)

    return new_im, mask_enlarged,new_size


##################################### Generate #############################################################################
def generate_image(prompt, steps, height, width):
    prompt = translate_text(prompt)
    payload = {'prompt': prompt,"neg_prompt": neg_prompt,"steps":steps, 'height': height, 'width': width}
    result =  send_api_call(payload,url_generate)
    decoded_img = decode_result(result)
    return decoded_img

##################################### Inpaint #############################################################################

def inpaint_image(image_and_mask, prompt, steps):
    # creating mask and encoding it
    background_image, mask_image, size = get_inpaint_image_and_mask_and_size(image_and_mask)
    encoded_background_string = encode_image(background_image)
    encoded_mask_string = encode_image(mask_image)
    prompt = translate_text(prompt)
    payload = {'input_image':encoded_background_string,'mask_image':encoded_mask_string, 'prompt': prompt,"neg_prompt": neg_prompt, "steps": steps, 'height': size[1], 'width': size[0]}
    result =  send_api_call(payload,url_inpaint_and_outpaint)
    decoded_img = decode_result(result)
    return decoded_img

##################################### Outpaint #############################################################################


def outpaint_image(image_and_mask,ratio, prompt, steps):
    # creating mask and encoding it
    background_image, mask_image, size = get_outpaint_image_and_mask_and_size(image_and_mask, ratio)
    encoded_background_string = encode_image(background_image)
    encoded_mask_string = encode_image(mask_image)
    prompt = translate_text(prompt)
    payload = {'input_image':encoded_background_string,'mask_image':encoded_mask_string, 'prompt': prompt,"neg_prompt": neg_prompt, "steps": steps, 'height': size[1], 'width': size[0]}
    result =  send_api_call(payload,url_inpaint_and_outpaint)
    decoded_img = decode_result(result)
    return decoded_img

##########################################################################################################################


import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.HTML("""
    <div style="text-align: center; padding: 20px;">
        <h1 style="font-size: 3em; color: #4A90E2;">✨ GenFillX ✨</h1>
        <p style="font-size: 1.2em; color: #666;">A simple yet powerful image generation, inpainting & outpainting tool using AI</p>
    </div>
    """)

    with gr.Tab("🎨 Generate"):
        with gr.Column():
            gr.Markdown("#### Generate Images from Prompts")
            demo1 = gr.Interface(
                fn=generate_image,
                inputs=[
                    gr.Text(label="Prompt", placeholder="Describe the image you want to generate..."),
                    gr.Number(label="Steps", value=5, precision=0),
                    gr.Number(label="Height", value=512, precision=0),
                    gr.Number(label="Width", value=512, precision=0)
                ],
                outputs=gr.Image(label="Generated Image", type="pil"),
            )

    with gr.Tab("🖌️ Inpaint"):
        with gr.Column():
            gr.Markdown("#### Fill missing parts of your image with AI")
            demo2 = gr.Interface(
                fn=inpaint_image,
                inputs=[
                    gr.ImageMask(label="Input Image (with mask)", type='pil', height=720, width=720),
                    gr.Text(label="Prompt", placeholder="Describe the content to fill in..."),
                    gr.Number(label="Steps", value=15, precision=0)
                ],
                outputs=gr.Image(label="Inpainted Image", type="pil"),
            )

    with gr.Tab("🧩 Outpaint"):
        with gr.Column():
            gr.Markdown("#### Expand your image beyond its borders")
            demo3 = gr.Interface(
                fn=outpaint_image,
                inputs=[
                    gr.Image(label="Input Image", height=720, width=720),
                    gr.Radio([25, 75, 100], label="Outpaint %", info="Choose how much to expand"),
                    gr.Text(label="Prompt", placeholder="Describe the extended parts..."),
                    gr.Number(label="Steps", value=15, precision=0)
                ],
                outputs=gr.Image(label="Outpainted Image", type="pil"),
            )

demo.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7153fbe05d0f69501.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
